## LinkedIn OAuth 2.0 Flow with IPython Notebook

LinkedIn implements OAuth 2.0 as one of its standard authentication mechanism, and "Example 3" demonstrates how to use it to get an access token for making API requests once you've created an app and gotten the "API Key" and "Secret Key" values that are part of the OAuth flow. Note that you will need an ordinary LinkedIn account in order to login, create an app, and get these credentials. You can create an app through the "Developer" section of your account settings as shown below or by navigating directly to https://www.linkedin.com/secure/developer.

You must ensure that your browser is not blocking popups in order for this script to work.

<img src="http://nbviewer.ipython.org/github/furukama/Mining-the-Social-Web-2nd-Edition/blob/master/ipynb/resources/ch03-linkedin/images/LinkedIn-app.png">


### Using LinkedIn OAuth credentials to receive an access token an authorize an application

Note: You must ensure that your browser is not blocking popups in order for this script to work. LinkedIn's OAuth flow appears to expressly involve opening a new window, and it does not appear that an inline frame can be used as is the case with some other social web properties. You may also find it very convenient to ensure that you are logged into LinkedIn at http://www.linkedin.com/ with this browser before executing this script, because the OAuth flow will prompt you every time you run it if you are not already logged in. If for some reason you cause IPython Notebook to hang, just select "Kernel => Interrupt" from its menu.

In [ ]:
import os
from threading import Timer
from flask import Flask, request
from linkedin import linkedin
from IPython.display import display
from IPython.display import Javascript as JS

# XXX: Get these values from your application's settings for the OAuth flow

CONSUMER_KEY = ''
CONSUMER_SECRET = ''

# This value is where LinkedIn will redirect. We'll configure an embedded
# web server to be serving requests here

RETURN_URL = 'http://ipython.ipeirotis.com:5000/oauth_helper'

# A temporary file to store a code from the web server
OAUTH_FILE = '/home/ubuntu/data/linkedin.authorization_code'

# Configure an emedded web server that accepts one request, stores a file
# that will need to be accessed outside of the request context, and 
# immediately shuts itself down

webserver = Flask("OAuthHelper")
@webserver.route("/oauth_helper")
def oauth_helper():
    code = request.args.get('code')
    f = open(OAUTH_FILE, 'w') # Store the code as a file
    f.write(code)
    f.close()
    shutdown_after_request = request.environ.get('werkzeug.server.shutdown')
    shutdown_after_request()
    return """<p>Handled redirect and extracted code <strong>%s</strong> 
                 for authorization</p>""" % (code,)

# Send an OAuth request to LinkedIn, handle the redirect, and display the access
# token that's included in the redirect for the user to copy and paste

auth = linkedin.LinkedInAuthentication(CONSUMER_KEY, CONSUMER_SECRET, RETURN_URL, 
                                       linkedin.PERMISSIONS.enums.values())

# Display popup after a brief delay to ensure that the web server is running and 
# can handle the redirect back from LinkedIn

Timer(1, lambda: display(JS("window.open('%s')" % auth.authorization_url))).start()

# Run the server to accept the redirect back from LinkedIn and capture the access
# token. This command blocks, but the web server is configured to shut itself down
# after it serves a request, so after the redirect occurs, program flow will continue

webserver.run(host='0.0.0.0')

# As soon as the request finishes, the web server shuts down and these remaining commands
# are executed, which exchange an authorization code for an access token. This process
# seems to need full automation because the authorization code expires very quickly.

auth.authorization_code = open(OAUTH_FILE).read()
auth.get_access_token()

# Prevent stale tokens from sticking around, which could complicate debugging
os.remove(OAUTH_FILE)


# How you can use the application to access the LinkedIn API...
app = linkedin.LinkedInApplication(auth)
print app.get_profile()